In [18]:
from fal import FalDbt
import os

In [19]:
cwd = os.getcwd()
faldbt = FalDbt(profiles_dir=cwd, project_dir=cwd)

faldbt.list_sources()
# [['results', 'ticket_data_sentiment_analysis']]

# faldbt.list_models()
# {
#   'zendesk_ticket_metrics': <RunStatus.Success: 'success'>, 
#   'stg_o3values': <RunStatus.Success: 'success'>, 
#   'stg_zendesk_ticket_data': <RunStatus.Success: 'success'>, 
#   'stg_counties': <RunStatus.Success: 'success'>
# }

[['airbnb', 'calendar'],
 ['airbnb', 'neighbourhoods'],
 ['airbnb', 'reviews'],
 ['airbnb', 'listings'],
 ['tpch', 'orders'],
 ['tpch', 'customer'],
 ['tpch', 'lineitem'],
 ['tpch', 'nation'],
 ['tpch', 'part'],
 ['tpch', 'partsupp'],
 ['tpch', 'region'],
 ['tpch', 'supplier']]

In [20]:
# ref() means the same thing to everyone in the data team(think: data engineers, data analysts, data scientists)
# no need to fiddle with ad hoc SQL to get the dataframe
sentiments = faldbt.ref('fct_reviews')
sentiments.tail(10)

,review_id,listing_id,reviewer_id,reviewer_name,comments,reviewed_at
279644,741312449.0,48470868.0,260616736.0,Efrain,It was a really nice place with plenty of thin...,2021-03-21
279645,742003523.0,48470868.0,385926306.0,Jacob,"For the price, this place is decent sized with...",2021-03-24
279646,743252269.0,48483623.0,392565524.0,Manu,"Beautiful, spacious, vintage San Francisco hom...",2021-03-28
279647,744384779.0,48549141.0,394480110.0,Jaime,Overall we really enjoyed staying here. Really...,2021-04-01
279648,744445540.0,48551949.0,298453619.0,Tomas,Yusuke's place was situated in a convenient lo...,2021-04-01
279649,745676116.0,48640926.0,354445688.0,Izaya,"great space and location, loved it!",2021-04-04
279650,741947692.0,48695555.0,330370836.0,Gary,Great communication,2021-03-24
279651,745710493.0,48695555.0,383982125.0,Rodolfo,"I really loved this place, for the price, for ...",2021-04-04
279652,746229646.0,48891710.0,394612900.0,Mohamed,Thanks Murat for hosting us for the weekend. T...,2021-04-05
279653,744728722.0,48907152.0,360780320.0,Lysette,Jim’s place was amazing! The staff was very we...,2021-04-02


In [21]:
# let's enrich the dataframe with the sentiment scores
from textblob import TextBlob
from textblob.classifiers import NaiveBayesClassifier

sentiments_sample = sentiments.tail(10)

sentiments_polarity = []
sentiments_subjectivity = []
for index, row in sentiments_sample.iterrows():
    text = row['comments']
    blob = TextBlob(text)
    sentiments_polarity.append(blob.sentiment.polarity)
    sentiments_subjectivity.append(blob.sentiment.subjectivity)


sentiments_sample['polarity'] = sentiments_polarity
sentiments_sample['subjectivity'] = sentiments_subjectivity

sentiments_sample

,review_id,listing_id,reviewer_id,reviewer_name,comments,reviewed_at,polarity,subjectivity
279644,741312449.0,48470868.0,260616736.0,Efrain,It was a really nice place with plenty of thin...,2021-03-21,0.600000,1.000000
279645,742003523.0,48470868.0,385926306.0,Jacob,"For the price, this place is decent sized with...",2021-03-24,0.171897,0.642241
279646,743252269.0,48483623.0,392565524.0,Manu,"Beautiful, spacious, vintage San Francisco hom...",2021-03-28,0.457500,0.762500
279647,744384779.0,48549141.0,394480110.0,Jaime,Overall we really enjoyed staying here. Really...,2021-04-01,0.158874,0.548052
279648,744445540.0,48551949.0,298453619.0,Tomas,Yusuke's place was situated in a convenient lo...,2021-04-01,0.255128,0.560897
279649,745676116.0,48640926.0,354445688.0,Izaya,"great space and location, loved it!",2021-04-04,0.837500,0.775000
279650,741947692.0,48695555.0,330370836.0,Gary,Great communication,2021-03-24,0.800000,0.750000
279651,745710493.0,48695555.0,383982125.0,Rodolfo,"I really loved this place, for the price, for ...",2021-04-04,0.256250,0.650000
279652,746229646.0,48891710.0,394612900.0,Mohamed,Thanks Murat for hosting us for the weekend. T...,2021-04-05,0.442727,0.627273
279653,744728722.0,48907152.0,360780320.0,Lysette,Jim’s place was amazing! The staff was very we...,2021-04-02,0.166667,0.558333
